In [84]:
import numpy as np
import pandas as pd
from scipy.stats import norm
from scipy.stats import f



from statsmodels.tsa.tsatools import lagmat
import statsmodels.api as sm



# from statsmodels.sandbox.regression.gmm import IV2SLS

# from statsmodels.sandbox.regression.gmm import IVRegressionResults
from linearmodels import IV2SLS



In [86]:
data = pd.read_excel('PS4data.xls')

### Problem (a)

In [89]:
data['time'] = range(1, len(data) + 1)

data['C'] = (data['real consumption of nondurables'] + data['real consumption of services']) / data['population']

data['Y'] = data['real consumption of nondurables'] / data['population']

lags = lagmat(data['Y'], maxlag=1, trim='both')
lags_df = pd.DataFrame(lags, columns=['L1.Y'])

data_trimmed = data.iloc[1:].reset_index(drop=True)
data_trimmed = pd.concat([data_trimmed, lags_df], axis=1)


lags = lagmat(data['C'], maxlag=4, trim='both')
lags_df = pd.DataFrame(lags, columns=['L1.C', 'L2.C', 'L3.C', 'L4.C'])

data_trimmed = data_trimmed.iloc[3:].reset_index(drop=True)
data_trimmed = pd.concat([data_trimmed, lags_df], axis=1)

In [91]:
X = data_trimmed[['L1.C', 'L2.C', 'L3.C', 'L4.C']]
X = sm.add_constant(X)  # Add intercept
y = data_trimmed['C']
model = sm.OLS(y, X).fit()


print(model.summary())

                            OLS Regression Results                            
Dep. Variable:                      C   R-squared:                       1.000
Model:                            OLS   Adj. R-squared:                  1.000
Method:                 Least Squares   F-statistic:                 2.094e+05
Date:                Mon, 18 Nov 2024   Prob (F-statistic):               0.00
Time:                        10:02:34   Log-Likelihood:                 1745.1
No. Observations:                 216   AIC:                            -3480.
Df Residuals:                     211   BIC:                            -3463.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2.439e-05   1.92e-05      1.267      0.2

In [93]:
f_test_result = model.f_test("L2.C = L3.C = L4.C = 0")

print("F-test result:")
print(f_test_result)

F-test result:
<F test: F=8.409996895528257, p=2.6364145736185046e-05, df_denom=211, df_num=3>


In [95]:
data_trimmed.head()

,year,quarter,real consumption of nondurables,real consumption of services,cpi,real disposable income,population,equally weighted average return on the New York Stock Exchange,time,C,Y,L1.Y,L1.C,L2.C,L3.C,L4.C
0,1949,1,451.8,482.6,23.866667,1087.8,103584.33,0.03338,5,0.009021,0.004362,0.004347,0.008991,0.008910,0.008914,0.008824
1,1949,2,452.7,485.7,23.866667,1091.3,103838.00,-0.08055,6,0.009037,0.004360,0.004362,0.009021,0.008991,0.008910,0.008914
2,1949,3,449.2,484.5,23.800000,1096.8,104127.33,0.13730,7,0.008967,0.004314,0.004360,0.009037,0.009021,0.008991,0.008910
3,1949,4,455.5,486.0,23.700000,1106.3,104427.67,0.12362,8,0.009016,0.004362,0.004314,0.008967,0.009037,0.009021,0.008991
4,1950,1,461.1,493.6,23.533333,1186.1,104733.33,0.05450,9,0.009116,0.004403,0.004362,0.009016,0.008967,0.009037,0.009021


### Problem (c)

In [98]:
lags = lagmat(data['Y'], maxlag=1, trim='both')
lags_df = pd.DataFrame(lags, columns=['L1.Y'])

data_trimmed_2 = data.iloc[1:].reset_index(drop=True)
data_trimmed_2 = pd.concat([data_trimmed_2, lags_df], axis=1)


lags = lagmat(data['C'], maxlag=6, trim='both')
lags_df = pd.DataFrame(lags, columns=['L1.C', 'L2.C', 'L3.C', 'L4.C', 'L5.C', 'L6.C'])

data_trimmed_2 = data_trimmed_2.iloc[5:].reset_index(drop=True)
data_trimmed_2 = pd.concat([data_trimmed_2, lags_df], axis=1)


data_trimmed_2['lgC'] = np.log(data_trimmed_2['C'] / data_trimmed_2['L1.C'])
data_trimmed_2['lgY'] = np.log(data_trimmed_2['Y'] / data_trimmed_2['L1.Y'])


data_trimmed_2['L2.lgC'] = np.log(data_trimmed_2['L2.C'] / data_trimmed_2['L3.C'])
data_trimmed_2['L3.lgC'] = np.log(data_trimmed_2['L3.C'] / data_trimmed_2['L4.C'])
data_trimmed_2['L4.lgC'] = np.log(data_trimmed_2['L4.C'] / data_trimmed_2['L5.C'])
data_trimmed_2['L5.lgC'] = np.log(data_trimmed_2['L5.C'] / data_trimmed_2['L6.C'])



In [100]:
### Construct regression formula

endog = data_trimmed_2['lgY']

instruments = data_trimmed_2[['L2.lgC', 'L3.lgC', 'L4.lgC', 'L5.lgC']].dropna()

data_trimmed_2["const"] = 1

results_2SLS = IV2SLS(dependent=data_trimmed_2['lgC'],exog = data_trimmed_2["const"], endog=endog, instruments=instruments).fit(cov_type="robust")


print(results_2SLS)


                          IV-2SLS Estimation Summary                          
Dep. Variable:                    lgC   R-squared:                      0.6818
Estimator:                    IV-2SLS   Adj. R-squared:                 0.6803
No. Observations:                 214   F-statistic:                    21.952
Date:                Mon, Nov 18 2024   P-value (F-stat)                0.0000
Time:                        10:02:35   Distribution:                  chi2(1)
Cov. Estimator:                robust                                         
                                                                              
                             Parameter Estimates                              
            Parameter  Std. Err.     T-stat    P-value    Lower CI    Upper CI
------------------------------------------------------------------------------
const          0.0026     0.0006     4.5679     0.0000      0.0015      0.0037
lgY            0.6969     0.1487     4.6853     0.00

### Problem (d)

In [82]:
model_1 = sm.OLS(data_trimmed_2['lgY'], data_trimmed_2[['const', 'L2.lgC', 'L3.lgC', 'L4.lgC', 'L5.lgC']]).fit()
data_trimmed_2['u'] = model_1.resid 
model_2 = sm.OLS(data_trimmed_2['lgC'], data_trimmed_2[['const', 'lgY', 'u']]).fit(cov_type="HC0", use_t=True )
print(model_2.summary())


                            OLS Regression Results                            
Dep. Variable:                    lgC   R-squared:                       0.725
Model:                            OLS   Adj. R-squared:                  0.723
Method:                 Least Squares   F-statistic:                     250.0
Date:                Mon, 18 Nov 2024   Prob (F-statistic):           2.20e-56
Time:                        09:46:29   Log-Likelihood:                 960.45
No. Observations:                 214   AIC:                            -1915.
Df Residuals:                     211   BIC:                            -1905.
Df Model:                           2                                         
Covariance Type:                  HC0                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0026      0.001      4.895      0.0

In [106]:
data_trimmed_2['resid_2SLS'] = results_2SLS.resids

model_3 = sm.OLS(data_trimmed_2['resid_2SLS'], data_trimmed_2[['const', 'L2.lgC', 'L3.lgC', 'L4.lgC', 'L5.lgC']]).fit()

print(model_3.summary())

                            OLS Regression Results                            
Dep. Variable:             resid_2SLS   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                  0.018
Method:                 Least Squares   F-statistic:                     1.956
Date:                Mon, 18 Nov 2024   Prob (F-statistic):              0.102
Time:                        10:04:55   Log-Likelihood:                 948.64
No. Observations:                 214   AIC:                            -1887.
Df Residuals:                     209   BIC:                            -1870.
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         -0.0002      0.000     -0.578      0.5